In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wikipedia
import xml.etree.ElementTree as ET
import re
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
from sklearn.metrics import r2_score

%matplotlib inline

In [2]:
sal_col = 'Salario mensual (AR$)'
df = pd.read_csv('sysarmy2016-1.csv')
df = df[df['Bruto o neto?'] == 'Bruto']
df = df[(df['Salario mensual (AR$)'].astype(float) > 100) & (df['Salario mensual (AR$)'].astype(float) <= 1_000_000)]
df.head()

,Timestamp,Soy,Tengo,Trabajo en,Años de experiencia,Años en el puesto actual,Trabajo de,Tecnologías que utilizás,Tecnologías que utilizás.1,Automation o funcional?,...,Qué tan conforme estás con tu sueldo?,Cómo creés que está tu sueldo con respecto a Julio 2015,Recibís algún tipo de bono,A qué está atado el bono,Tuviste ajustes por inflación en 2015?,De qué % fue el ajuste anual?,Cantidad de empleados,Cambiaste de empresa en los últimos 6 meses?,Cuál fue el principal motivo de cambio?,Beneficios extra
1,1/11/2016 9:04:18,Hombre,31 - 33,Ciudad Autónoma de Buenos Aires,8 - 10,5 - 6,SysAdmin / DevOps,"Linux, Windows, AWS / Rackspace, Docker / Cont...",NaN,NaN,...,3,2,De uno a tres sueldos,Mix de las anteriores,Uno,10.0,11-50,No,No cambié de empresa,"Bonos por desempeño, Horarios flexibles, Lapto..."
8,1/11/2016 9:08:26,Hombre,31 - 33,Ciudad Autónoma de Buenos Aires,8 - 10,Menos de un año,SysAdmin / DevOps,"Windows, VMWare",NaN,NaN,...,3,3,No,No recibo bono,Dos,29.0,51-100,Sí,DESPIDO,"Abono de celular y/o Internet, Laptop"
9,1/11/2016 9:08:57,Hombre,34 - 36,Ciudad Autónoma de Buenos Aires,10+,5 - 6,Middleware,NaN,NaN,NaN,...,4,2,No,No recibo bono,Dos,20.0,1001+,No,No cambié de empresa,"Abono de celular y/o Internet, Aumento por inf..."
10,1/11/2016 9:10:29,Hombre,41 - 45,Santa Fe,10+,10+,Infosec,NaN,NaN,NaN,...,3,3,No,No recibo bono,No,32.0,1001+,No,No cambié de empresa,Abono de celular y/o Internet
12,1/11/2016 9:12:26,Hombre,27 - 30,Ciudad Autónoma de Buenos Aires,5 - 7,3 - 4,SysAdmin / DevOps,"Linux, AWS / Rackspace",NaN,NaN,...,4,4,De uno a tres sueldos,Performance individual,Uno,15.0,201-500,No,No cambié de empresa,"Abono de celular y/o Internet, Bonos por desem..."


In [3]:
h, m = (
df[df['Soy'] == 'Hombre']['Salario mensual (AR$)'].median(),
df[df['Soy'] == 'Mujer']['Salario mensual (AR$)'].median(),
)
(h - m) / h

0.03125

In [4]:
df.columns

Index(['Timestamp', 'Soy', 'Tengo', 'Trabajo en', 'Años de experiencia',
       'Años en el puesto actual', 'Trabajo de', 'Tecnologías que utilizás',
       'Tecnologías que utilizás.1', 'Automation o funcional?',
       'Tecnologías que utilizás.2', 'Tecnologías que utilizás.3',
       'Tenés guardias?', 'Cuánto cobrás por guardia (AR$)',
       'Porcentaje, bruto o neto?', 'Tipo de contrato',
       'Salario mensual (AR$)', 'Bruto o neto?',
       'Qué tan conforme estás con tu sueldo?',
       'Cómo creés que está tu sueldo con respecto a Julio 2015',
       'Recibís algún tipo de bono', 'A qué está atado el bono',
       'Tuviste ajustes por inflación en 2015?',
       'De qué % fue el ajuste anual?', 'Cantidad de empleados',
       'Cambiaste de empresa en los últimos 6 meses?',
       'Cuál fue el principal motivo de cambio?', 'Beneficios extra'],
      dtype='object')

In [5]:
best = {'colsample_bytree': 0.7000000000000001, 'gamma': 0.8500000000000001, 'learning_rate': 0.025, 'max_depth': 16, 'min_child_weight': 15.0, 'n_estimators': 175, 'subsample': 0.8099576733552297}
regions_map = {
    'Ciudad Autónoma de Buenos Aires': 'AMBA',
    'GBA': 'AMBA',
    'Catamarca': 'NOA',
    'Chaco': 'NEA',
    'Chubut': 'Patagonia',
    'Corrientes': 'NEA',
    'Entre Ríos': 'NEA',
    'Formosa': 'NEA',
    'Jujuy': 'NOA',
    'La Pampa': 'Pampa',
    'La Rioja': 'NOA',
    'Mendoza': 'Cuyo',
    'Misiones': 'NEA',
    'Neuquén': 'Patagonia',
    'Río Negro': 'Patagonia',
    'Salta': 'NOA',
    'San Juan': 'Cuyo',
    'San Luis': 'Cuyo',
    'Santa Cruz': 'Patagonia',
    'Santa Fe': 'Pampa',
    'Santiago del Estero': 'NOA',
    'Tucumán': 'NOA',
    'Córdoba': 'Pampa',
    'Provincia de Buenos Aires': 'Pampa',
    'Tierra del Fuego': 'Patagonia',
}
class Model:
    def __init__(self, **params):
        self.regressor_ = xgb.XGBRegressor(**params)

    def get_params(self, deep=True):
        return self.regressor_.get_params(deep=deep)

    def set_params(self, **params):
        return self.regressor_.set_params(**params)
    
    def clean_words(self, field, value):
        value = value.replace('Microsoft Azure (Tables, CosmosDB, SQL, etc)', 'Microsoft Azure(TablesCosmosDBSQLetc)')
        value = value.replace('Snacks, golosinas, bebidas', 'snacks')
        value = value.replace('Descuentos varios (Clarín 365, Club La Nación, etc)', 'descuentos varios')
        value = value.replace('Sí, de forma particular', 'de forma particular')
        value = value.replace('Sí, los pagó un empleador', 'los pagó un empleador')
        value = value.replace('Sí, activa', 'activa')
        value = value.replace('Sí, pasiva', 'pasiva')
        return [self.clean_word(field, v) for v in value.split(',') if self.clean_word(field, v)]

    def clean_word(self, field, word):
        val = str(word).lower().strip().replace(".", "")
        if val in ('ninguno', 'ninguna', 'no', '0', 'etc)', 'nan'):
            return ''
        if field == 'Lenguajes de programación' and val == 'Microsoft Azure(TablesCosmosDBSQLetc)':
            return 'Microsoft Azure (Tables, CosmosDB, SQL, etc)'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('pycon', 'pyconar'):
            return 'pyconar'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('nodeconf', 'nodeconfar'):
            return 'nodeconfar'
        if field == '¿A qué eventos de tecnología asististe en el último año?' and val in ('meetup', 'meetups'):
            return 'meetups'
        if field == '¿A qué eventos de tecnología asististe en el último año?':
            return val.replace(' ', '')
        if field == 'Beneficios extra' and val == 'snacks':
            return 'snacks, golosinas, bebidas'
        if field == 'Beneficios extra' and val == 'descuentos varios':
            return 'descuentos varios (clarín 365, club la nación, etc)'
        return val

    def row_to_words(self, row):
        return [
            f'{key}={row.fillna("")[key]}'
            for key
            in (
                'Soy',
                'Trabajo de',
                'Tipo de contrato',
                'Cantidad de empleados',
            )
        ] + [
            f'{k}={v}' for k in (
                'Tenés guardias?',
                'Beneficios extra',
                'Tecnologías que utilizás',
                'Tecnologías que utilizás.1',
                'Tecnologías que utilizás.2',
                'Tecnologías que utilizás.3',
            ) for v in self.clean_words(k, row.fillna('')[k])
        ] + [
            f'region={regions_map[row["Trabajo en"]]}'
        ]

    def encode_row(self, row):
        ws = self.row_to_words(row)
        row = pd.Series([w in ws for w in self.valid_words_] + [
            int(row['Años de experiencia'].replace('+', '').replace('Menos', '0').split(' ')[0]),
            int(row['Tengo'].replace('+', '').replace('Menos', '0').split(' ')[0]),
        ])
        return row

    def fit(self, X, y, **params):
        counts = {}
        for i in range(X.shape[0]):
            for word in self.row_to_words(X.iloc[i]):
                counts[word] = counts.get(word, 0) + 1
        self.valid_words_ = [word for word, c in counts.items() if c > 0.01*X.shape[0]]
        self.regressor_.fit(X.apply(self.encode_row, axis=1).astype(float), y, **params)
        return self
   
    def predict(self, X):
        return self.regressor_.predict(X.apply(self.encode_row, axis=1).astype(float))
    
    def score(self, X, y):
        return r2_score(y, self.predict(X))

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=99)
kf_models = []
for train_index, test_index in kf.split(df):
    model = Model(**best).fit(df.iloc[train_index], df.iloc[train_index]['Salario mensual (AR$)'].astype(float))
    df.loc[df.index[test_index], 'e(salary)'] = model.predict(df.iloc[test_index])
    df['Soy'] = df['Soy'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    df.loc[df.index[test_index], 'e_gr(salary)'] = model.predict(df.iloc[test_index])
    df['Soy'] = df['Soy'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    kf_models.append(model)
df['e_h(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Soy'] == 'Hombre' else row['e_gr(salary)'], axis=1)
df['e_m(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Soy'] == 'Mujer' else row['e_gr(salary)'], axis=1)
df['e_g_diff(salary)'] = (df['e_h(salary)'] - df['e_m(salary)']) / df['e_h(salary)']
r2_score(df[sal_col], df['e(salary)'])

0.07711454283304686

In [7]:
df['e_diff(salary)'] = (df['Salario mensual (AR$)'].astype(float) - df['e(salary)']) / df['Salario mensual (AR$)'].astype(float)
df = df[~df['e_diff(salary)'].apply(lambda x: x > 5 or x < -5)]

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=99)
kf_models = []
for train_index, test_index in kf.split(df):
    model = Model(**best).fit(df.iloc[train_index], df.iloc[train_index]['Salario mensual (AR$)'].astype(float))
    df.loc[df.index[test_index], 'e(salary)'] = model.predict(df.iloc[test_index])
    df['Soy'] = df['Soy'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    df.loc[df.index[test_index], 'e_gr(salary)'] = model.predict(df.iloc[test_index])
    df['Soy'] = df['Soy'].apply(lambda g: {'Hombre': 'Mujer', 'Mujer': 'Hombre'}[g])
    kf_models.append(model)
df['e_h(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Soy'] == 'Hombre' else row['e_gr(salary)'], axis=1)
df['e_m(salary)'] = df.apply(lambda row: row['e(salary)'] if row['Soy'] == 'Mujer' else row['e_gr(salary)'], axis=1)
df['e_g_diff(salary)'] = (df['e_h(salary)'] - df['e_m(salary)']) / df['e_h(salary)']
r2_score(df[sal_col], df['e(salary)'])

0.08848302137815456

In [9]:
df['e_g_diff(salary)'].median()

0.029420825483034936